# Using Yoneda to Map Without a Functor Instance
###### (https://gist.github.com/Icelandjack/aecf49b75afcfcee9ead29d27cc234d5)

A type class for a list implementation

In [15]:
class List f where
  nil :: f a
  cons :: a -> f a -> f a

The `Yoneda` lemma:

In [10]:
:ext RankNTypes

newtype Yoneda f a = Yoneda { runYoneda :: forall b. (a -> b) -> f b } 

liftYoneda :: Functor f => f a -> Yoneda f a
liftYoneda a = Yoneda $ \f -> fmap f a

lowerYoneda :: Functor f => Yoneda f a -> f a
lowerYoneda y = runYoneda y id

We define a list like this

In [12]:
abc :: List f => f Char
abc = 'a' `cons` ('b' `cons` ('c' `cons` nil))

We don't need a `Functor` instance to map over `abc`. 
Make a list instance from Yoneda. Think of `Yoneda F` as `F` in the process of being mapped over:

In [19]:
instance List f => List (Yoneda f) where
  nil = Yoneda (const nil)
  cons a (Yoneda abfb) = Yoneda $ \cont ->
    cont a `cons` abfb cont

Then we can map like this:

In [28]:
:ext FlexibleContexts

:m Data.Char

ordAbc = runYoneda abc ord
:t ordAbc

show (ordAbc :: [Int])

ordAbc :: forall (f :: * -> *). List (Yoneda f) => f Int

<interactive>:1:7: error:
    • No instance for (Ghci2.List (Yoneda [])) arising from a use of ‘ordAbc’
    • In the first argument of ‘show’, namely ‘(ordAbc :: [Int])’
      In the expression: show (ordAbc :: [Int])
      In an equation for ‘it’: it = show (ordAbc :: [Int])